In [1]:
import os
import torch
import json

from bcnf.utils import get_dir, load_config, sub_root_path
from bcnf.train import Trainer
from bcnf import CondRealNVP_v2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
MODEL_NAME = 'videos_CNN_LSTM_large'

In [3]:
config_path_pattern = os.path.join("{{BCNF_ROOT}}", "configs", "runs", f"{MODEL_NAME}.yaml")
config_path = sub_root_path(config_path_pattern)
print(f'Loading config from {config_path}')

config = load_config(config_path, verify=False)

Loading config from /home/psaegert/Projects/bcnf/configs/runs/videos_CNN_LSTM_large.yaml


In [4]:
model = CondRealNVP_v2.from_config(config).to(device)

print(f'{model.n_params:,}')

67,787,515


In [5]:
test = torch.randn(5,2,30,90,160)
test = test.to(device)
out = model.feature_network.forward(test)
print(out.shape)

torch.Size([5, 30, 1000])


In [6]:
trainer = Trainer(
    config={k.lower(): v for k, v in config.to_dict().items()},
    project_name="bcnf-test",
    parameter_index_mapping=model.parameter_index_mapping,
    verbose=True,
)

Using dtype: torch.float32
Loading data from /home/computerman/Desktop/IGNNS/IGNNS-final-project/data/bcnf-data/fixed_data_render_2s_15FPS/train...


Loading data from directory: 100%|██████████| 1/1 [00:02<00:00,  2.78s/it, file=fixed_data_render_2s_15FPS_1.pkl]


Using videos data for training. Shapes:
Conditions: [torch.Size([1000, 2, 30, 90, 160]), torch.Size([1000, 7])]
Parameters: torch.Size([1000, 19])


In [7]:
model = trainer.train(model)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ichich111222333 (balisticcnf). Use `wandb login --relogin` to force relogin


Train: -0.5251 - Val: -0.4199 (avg: 0.6378, min: 0.7824) | lr: 2.00e-04 - Patience: 2/500 - z: (-0.0206 ± 0.3233) ± (0.8837 ± 0.2210):   0%|          | 85/50000 [04:42<46:01:34,  3.32s/it] 
Traceback (most recent call last):
  File "/home/psaegert/Projects/bcnf/src/bcnf/train/trainer.py", line 100, in train
    model = self._train(
            ^^^^^^^^^^^^
  File "/home/psaegert/Projects/bcnf/src/bcnf/train/trainer.py", line 228, in _train
    loss = self._train_batch(y, *conditions, model=model, optimizer=optimizer, loss_function=loss_function)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/psaegert/Projects/bcnf/src/bcnf/train/trainer.py", line 301, in _train_batch
    z = model.forward(
        ^^^^^^^^^^^^^^
  File "/home/psaegert/Projects/bcnf/src/bcnf/models/cnf.py", line 497, in forward
    self.log_det_J = torch.zeros(y.shape[0]).to(self.device)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

distance_to_last_best_val_loss_fold_-1,▁▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅████▅▅█
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr_fold_-1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
time_fold_-1,▁▁▁▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅███████████████████
train_loss_fold_-1,█▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss_fold_-1,█▅▅▄▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
z_mean_mean_fold_-1,▃▅▅▅▅▇▄▄▄▆▃▄▅▄▄▃▇▆▃▆▅▆▅▆▇▅█▄▅▄▄▄▅▂▅▄▄▁▆▅
z_mean_std_fold_-1,▃▂▂▄▅▇▆▅▁▇█▆▂▁▃▄▃▆▁▄▅▅▅▆▃▂▃▅▄▇▃▇▅▄▄▆▄▆▅▃
z_std_mean_fold_-1,▆▂▇▃▆▅▂▃▅▅▁█▆▅▄▅▃▄▃▄▃▇▄▄▃▂▃▆▆▅▄▅▂▆▂▆▃▅▃▅
z_std_std_fold_-1,█▃▅▁▅▂▃▄▅▆▃▇▇▆▄▅▅▃▅▅▃▃▆▃▃▄▄▇▄▆▄▆▄▄▃▆▅▆▅▃
distance_to_last_best_val_loss_fold_-1,2


KeyboardInterrupt: 

In [9]:
torch.save(model.state_dict(), os.path.join(get_dir('models', 'bcnf-models', MODEL_NAME, create=True), f"state_dict.pt"))

with open(os.path.join(get_dir('models', 'bcnf-models', MODEL_NAME, create=True), 'config.json'), 'w') as f:
    json.dump({'config_path': config_path_pattern}, f)

print(f"Model saved to {get_dir('models', 'bcnf-models', MODEL_NAME)}")

Model saved to /home/psaegert/Projects/bcnf/models/bcnf-models/videos_CNN_LSTM_large
